In [ ]:
# Cell 1
from google.colab import drive
drive.mount('/content/drive')

# Change to directory
%cd '/content/drive/MyDrive/CSE 8803 IUC project'

!pip install tweetnlp
!pip install pandas

In [ ]:
import tweetnlp
import pandas as pd
import datetime
import numpy as np

In [ ]:
location = 'Los Angeles'

In [ ]:
df = pd.read_csv('data/' + location + '.csv')

In [ ]:
df

In [ ]:
def analyze_time_frame(model, df, feat, task):
  dates = pd.unique(df['date'])
  analysis_list = []

  for date in dates:
    data_batch = df[df['date'] == date]
    if task == 'sentiment':
      results_batch = model.sentiment(data_batch['content'])
    elif task == 'irony':
      results_batch = model.irony(data_batch['content'])
    elif task == 'hate':
      results_batch = model.hate(data_batch['content'])
    elif task == 'offensive':
      results_batch = model.offensive(data_batch['content'])
    tuple_batch = []
    tuple_batch.append(date)
    for f in feat:
      temp = (pd.DataFrame(results_batch)['label'] == f)
      tuple_batch.append(temp.mean(axis=0))
    analysis_list.append(tuple_batch)
  
  return analysis_list

## Sentiment Analysis

In [ ]:
model_sentiment = tweetnlp.load_model('sentiment')

In [ ]:
feat_sentiment = ['positive', 'negative', 'neutral']
result_sentiment = analyze_time_frame(model_sentiment, df, feat_sentiment, 'sentiment')
result_sentiment = pd.DataFrame(result_sentiment, columns=['date']+feat_sentiment).set_index('date')

## Irony Detection

In [ ]:
model_irony = tweetnlp.load_model('irony')

In [ ]:
feat_irony = ['non_irony', 'irony']
result_irony = analyze_time_frame(model_irony, df, feat_irony, 'irony')
result_irony = pd.DataFrame(result_irony, columns=['date']+feat_irony).set_index('date')

## Hate Speech Detection

In [ ]:
model_hate = tweetnlp.load_model('hate')

In [ ]:
feat_hate = ['non-hate', 'hate']
result_hate = analyze_time_frame(model_hate, df, feat_hate, 'hate')
result_hate = pd.DataFrame(result_hate, columns=['date']+feat_hate).set_index('date')

## Offensive Language Detection

In [ ]:
model_off = tweetnlp.load_model('offensive')

In [ ]:
feat_off = ['non-offensive', 'offensive']
result_off = analyze_time_frame(model_off, df, feat_off, 'offensive')
result_off = pd.DataFrame(result_off, columns=['date']+feat_off).set_index('date')

In [ ]:
result = pd.concat((result_sentiment, result_irony, result_hate, result_off), axis=1)
result.to_csv('data/' + location + '_feat.csv')